In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output

In [3]:
dataset=pd.read_csv('station_day.csv')

In [4]:
dataset= dataset.drop(['StationId','Date'],axis=1)

In [5]:
dataset = dataset.dropna()

In [6]:
X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values

In [7]:
dataset.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
1,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.0,Moderate
2,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.0,Moderate
3,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.0,Moderate
4,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.0,Moderate
5,72.47,114.84,5.23,23.20,16.59,12.25,0.16,10.55,109.74,0.21,4.71,0.08,173.0,Moderate


In [26]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
dataset['AQI_Bucket'] = labelencoder.fit_transform(dataset['AQI_Bucket'])
dataset.head(100)

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
1,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.0,1
2,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.0,1
3,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.0,1
4,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.0,1
5,72.47,114.84,5.23,23.20,16.59,12.25,0.16,10.55,109.74,0.21,4.71,0.08,173.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,33.08,83.16,5.37,18.83,14.38,14.52,0.72,8.98,23.63,0.05,2.04,0.10,92.0,3
99,29.70,77.45,4.45,17.33,13.01,12.72,0.65,9.71,29.26,0.05,1.85,0.10,80.0,3
100,38.67,82.67,4.45,22.81,15.74,12.41,0.66,12.63,22.90,0.06,2.18,0.11,88.0,3
101,30.04,83.07,11.58,24.74,22.65,12.52,0.78,8.98,17.81,0.05,1.91,0.21,83.0,3


In [8]:
dataset['AQI_Bucket'].value_counts()

1    4402
3    3730
0    1073
2     610
5     399
4     100
Name: AQI_Bucket, dtype: int64

In [9]:
dataset.columns

Index(['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3',
       'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')

In [10]:
X = dataset[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3',
       'Benzene', 'Toluene', 'Xylene']]

In [11]:
X

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene
1,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06
2,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08
3,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12
4,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07
5,72.47,114.84,5.23,23.20,16.59,12.25,0.16,10.55,109.74,0.21,4.71,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...
106143,80.93,174.84,82.96,56.74,139.70,29.32,1.18,9.36,15.96,53.71,51.38,1.30
106144,103.75,217.89,139.58,63.28,181.55,27.56,1.71,11.43,24.75,53.58,55.10,2.29
106145,115.40,225.76,193.19,70.41,213.77,35.25,2.11,10.49,22.11,58.61,61.42,3.20
106146,97.37,179.87,105.12,64.34,169.46,36.24,1.51,6.90,12.55,64.90,62.12,2.16


In [12]:
y = dataset['AQI_Bucket']
y

1         1
2         1
3         1
4         1
5         1
         ..
106143    1
106144    2
106145    2
106146    2
106147    2
Name: AQI_Bucket, Length: 10314, dtype: int32

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [15]:
randomForest = RandomForestClassifier(bootstrap = False,
                                      max_depth = None,
                                      max_features = 'sqrt',
                                      min_samples_leaf = 1,
                                      min_samples_split = 5,
                                      n_estimators = 25)
randomForest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_split=5, n_estimators=25)

In [16]:
randomForest_predict = randomForest.predict(X_test)

In [17]:
randomForestAccuracy = round(accuracy_score(randomForest_predict, y_test) * 100, 2)
kFold = KFold(n_splits = 10, random_state = 10)
randomForestCVAccuracy = cross_val_score(randomForest, X, y, cv = 10, scoring = 'accuracy')

print("The accuracy of Random Forest Classifier is : ", randomForestAccuracy)
print("The cross validated score for Random Forest Classifier is : ",round(randomForestCVAccuracy.mean() * 100, 2))

The accuracy of Random Forest Classifier is :  83.52
The cross validated score for Random Forest Classifier is :  76.3


In [18]:
import joblib

joblib.dump(randomForest,'AirQuality.sav')

['AirQuality.sav']